<a href="https://colab.research.google.com/github/deep1003/deep1003/blob/master/PatentBERT01.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Step 1: 데이터 로딩 및 정제

# Google Drive를 마운트하여 파일 접근
from google.colab import drive
import pandas as pd

# Google Drive 마운트
drive.mount('/content/drive')

# Google Drive의 CSV 파일 경로 (경로를 적절히 업데이트하세요)
file_path = '/content/drive/MyDrive/data/patents.csv'

# CSV 파일 불러오기
df = pd.read_csv(file_path)

# 데이터가 성공적으로 로드되었는지 확인
print("데이터 로딩 성공!")
print(df.head())


Mounted at /content/drive


ParserError: Error tokenizing data. C error: Expected 8 fields in line 91, saw 9


In [ ]:

# 기본 텍스트 정제 함수 정의
import re

def clean_text(text):
    # 비알파벳 문자 제거 및 소문자로 변환
    text = re.sub(r'[^a-zA-Z\s]', '', text)
    text = text.lower().strip()
    return text

# 'abstract' 열에 텍스트 정제 적용
df['abstract_clean'] = df['abstract'].apply(clean_text)

# 정제된 데이터 확인
print("정제된 초록:")
print(df[['abstract', 'abstract_clean']].head())


In [ ]:
# Step 2: 벡터 임베딩 특징 추출

import torch
from transformers import BertTokenizer, BertModel

# PatentBERT 모델 및 토크나이저 불러오기
tokenizer = BertTokenizer.from_pretrained("anferico/bert-for-patents")
model = BertModel.from_pretrained("anferico/bert-for-patents")

# 벡터 임베딩 추출 함수 정의
def extract_embeddings(text):
    # 입력 텍스트를 BERT가 처리할 수 있는 형식으로 토크나이징
    inputs = tokenizer(text, return_tensors="pt", truncation=True, padding=True, max_length=128)

    # PatentBERT를 사용하여 임베딩 추출
    with torch.no_grad():
        outputs = model(**inputs)

    # [CLS] 토큰의 벡터 임베딩을 반환
    cls_embedding = outputs.pooler_output.squeeze().numpy()
    return cls_embedding

# 모든 초록에 대해 벡터 임베딩 추출
df['embeddings'] = df['abstract_clean'].apply(extract_embeddings)

# 추출된 임베딩 확인
print("임베딩 추출 완료:")
print(df['embeddings'].head())


In [ ]:
# Step 3: 분류, 검증 및 시각화

import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
import matplotlib.pyplot as plt
import seaborn as sns
import torch.nn as nn
import torch.optim as optim

# 분류를 위한 데이터 준비
# 임베딩을 numpy 배열로 변환
X = np.vstack(df['embeddings'].values)
y = df['label']

# 레이블을 정수로 인코딩
label_encoder = LabelEncoder()
y_encoded = label_encoder.fit_transform(y)

# 데이터셋을 학습 세트와 검증 세트로 분할
X_train, X_val, y_train, y_val = train_test_split(X, y_encoded, test_size=0.2, random_state=42)

# 간단한 신경망 모델 정의 (분류용)
class SimpleNN(nn.Module):
    def __init__(self, input_size, num_classes):
        super(SimpleNN, self).__init__()
        self.fc1 = nn.Linear(input_size, 128)
        self.relu = nn.ReLU()
        self.fc2 = nn.Linear(128, num_classes)

    def forward(self, x):
        x = self.fc1(x)
        x = self.relu(x)
        x = self.fc2(x)
        return x

# 모델, 손실 함수 및 최적화기 초기화
input_size = X_train.shape[1]
num_classes = len(label_encoder.classes_)
model = SimpleNN(input_size, num_classes)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# 데이터를 torch 텐서로 변환
X_train_tensor = torch.tensor(X_train, dtype=torch.float32)
y_train_tensor = torch.tensor(y_train, dtype=torch.long)
X_val_tensor = torch.tensor(X_val, dtype=torch.float32)
y_val_tensor = torch.tensor(y_val, dtype=torch.long)

# 모델 학습 함수
def train_model(model, X_train_tensor, y_train_tensor, epochs=10):
    for epoch in range(epochs):
        model.train()  # 모델을 학습 모드로 전환
        optimizer.zero_grad()  # 이전 단계의 기울기 초기화
        outputs = model(X_train_tensor)  # 모델에 입력 데이터 전달
        loss = criterion(outputs, y_train_tensor)  # 손실 계산
        loss.backward()  # 역전파 수행
        optimizer.step()  # 가중치 업데이트
        print(f"에폭 {epoch+1}/{epochs}, 손실: {loss.item()}")

train_model(model, X_train_tensor, y_train_tensor)

# 모델 검증 함수
def evaluate_model(model, X_val_tensor, y_val_tensor):
    model.eval()  # 모델을 평가 모드로 전환
    with torch.no_grad():
        outputs = model(X_val_tensor)
        _, preds = torch.max(outputs, 1)  # 예측 결과 추출
        accuracy = accuracy_score(y_val_tensor.numpy(), preds.numpy())
        print(f"검증 정확도: {accuracy}")
        print("분류 보고서:")
        print(classification_report(y_val_tensor.numpy(), preds.numpy(), target_names=label_encoder.classes_))
        return y_val_tensor.numpy(), preds.numpy()

y_true, y_pred = evaluate_model(model, X_val_tensor, y_val_tensor)

# 시각화: 혼동 행렬
cm = confusion_matrix(y_true, y_pred)
plt.figure(figsize=(8, 6))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', xticklabels=label_encoder.classes_, yticklabels=label_encoder.classes_)
plt.xlabel('예측값')
plt.ylabel('실제값')
plt.title('혼동 행렬')
plt.show()
